In [2]:
import pandas as pd 

In [3]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [3]:
# aggregate = [
#   {
#     "$match": {
#       "name": {
#         "$regex": "single-plane-ophys",  # add .*processed or .*raw to narrow search 
#         "$options": "i"
#       },
#       "data_description.project_name": "Brain Computer Interface",
#       "session.session_type": "BCI single neuron stim"
#     }
#   },
#   {
#     "$project": {
#       "name": 1,
#       "subject_id": "$data_description.subject_id",
#       "session_time": "$session.session_start_time", 
#       "session_type": "$session.session_type", 
#       "genotype": "$subject.genotype", 
#       "virus_injection_materials": "$procedures.subject_procedures.procedures.injection_materials.name",
#       "session_number": {
#         "$filter": {
#           "input": "$session.stimulus_epochs",
#           "as": "epoch",
#           "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
#         }
#       }
#     }
#   },
#   {
#     "$project": {
#       "name": 1,
#       "subject_id": 1,
#       "session_time": 1, 
#       "session_type": 1,
#       "genotype": 1,
#       "virus_injection_materials": 1,
#       "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] }
#     }
#   },
#   {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}, 
#   {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}
# ]

# records = docdb_api_client.aggregate_docdb_records(
#     pipeline = aggregate,
# )

In [9]:
aggregate = [
  {
    "$match": {
      "name": {
        "$regex": "single-plane-ophys",  # add .*processed or .*raw to narrow search 
        "$options": "i"
      },
      "data_description.project_name": "Brain Computer Interface",
      "session.session_type": "BCI single neuron stim"
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": "$data_description.subject_id",
      "session_time": "$session.session_start_time", 
      "session_type": "$session.session_type", 
      "genotype": "$subject.genotype", 
      "virus_injection_materials": "$procedures.subject_procedures.procedures.injection_materials.name",
      "session_number": {
        "$filter": {
          "input": "$session.stimulus_epochs",
          "as": "epoch",
          "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
        }
      },
      "ophy_fov": {
        "$filter": {
          "input": "$session.data_streams",
          "as": "stream",
          "cond": { "$ne": ["$$stream.ophy_fov", "test"] }
        }
      }
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": 1,
      "session_time": 1, 
      "session_type": 1,
      "genotype": 1,
      "virus_injection_materials": 1,
      "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] },
      "ophy_fov": { "$arrayElemAt": ["$ophy_fov.ophy_fov", 0] }
    }
  },
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}, 
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)

In [10]:
records

[{'_id': '4993b84b-3fd3-4dc4-90ab-2a7e7d4d5f07',
  'name': 'single-plane-ophys_772414_2025-01-28_16-19-17',
  'subject_id': '772414',
  'session_time': '2025-01-28T14:55:51.847000-08:00',
  'session_type': 'BCI single neuron stim',
  'genotype': 'Camk2a-tTA/wt;tetO-GCaMP6s/wt',
  'virus_injection_materials': ['pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413'],
  'session_number': 6},
 {'_id': '8caa31e1-1ddb-41d6-a8cc-b271f6dc3fc5',
  'name': 'single-plane-ophys_740369_2025-01-09_17-18-37_processed_2025-01-10_10-00-37',
  'subject_id': '740369',
  'session_time': '2025-01-09T16:01:04.455000-08:00',
  'session_type': 'BCI single neuron stim',
  'genotype': 'Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-ICL-ChRmine-oScarlet-IRES2-tTA2-WPRE)-hyg/wt',
  'virus_injection_materials': ['pAAV-hSyn1-RiboL1-GCaMP8s-WPRE'],
  'session_number': 22},
 {'_id': '72c81192-bf3e-4d7d-9a8e-22847f13fca4',
  'name': 'single-plane-ophys_772414_2025-01-24_14-49-14_processed_2025-01-25_04-46-25',
  'subject_id':

In [6]:
name = [] 
subject_id = [] 
session_time = [] 
session_type = [] 
genotype = []
virus = []
session_number = []

for record in records: 
    name.append(record["name"])
    subject_id.append(record["subject_id"])
    session_time.append(record["session_time"])
    session_type.append(record["session_type"]) 
    genotype.append(record["genotype"])
    virus.append(record["virus_injection_materials"][0])
    session_number.append(record["session_number"])
    
    
bci_df = pd.DataFrame() 
bci_df["name"] = name 
bci_df["subject_id"] = subject_id
bci_df["session_time"] = session_time
bci_df["session_type"] = session_type
bci_df["genotype"] = genotype
bci_df["virus"] = virus
bci_df["session_number"] = session_number

In [7]:
bci_df

,name,subject_id,session_time,session_type,genotype,virus,session_number
0,single-plane-ophys_772414_2025-01-28_16-19-17,772414,2025-01-28T14:55:51.847000-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,6.0
1,single-plane-ophys_740369_2025-01-09_17-18-37_...,740369,2025-01-09T16:01:04.455000-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,22.0
2,single-plane-ophys_772414_2025-01-24_14-49-14_...,772414,2025-01-24T13:30:13.734999-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,4.0
3,single-plane-ophys_772414_2025-01-23_16-03-51,772414,2025-01-23T14:40:12.962999-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,3.0
4,single-plane-ophys_772414_2025-01-28_16-19-17_...,772414,2025-01-28T14:55:51.847000-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,6.0
...,...,...,...,...,...,...,...
102,single-plane-ophys_767715_2025-02-06_18-59-24_...,767715,2025-02-06T18:59:24.815999-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,4.0
103,single-plane-ophys_754303_2025-01-27_20-01-31,754303,2025-01-27T18:46:54.109999-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,5.0
104,single-plane-ophys_766719_2025-01-27_18-25-39_...,766719,2025-01-27T17:02:50.868000-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,5.0
105,single-plane-ophys_772414_2025-01-27_15-57-05_...,772414,2025-01-27T14:23:26.693000-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,5.0


In [7]:
bci_df.to_csv('/data/metadata.csv', index= False) 